In [1]:
from utils import get_puzzle_input_as_rows


In [15]:
from typing import Iterable

def window(seq, window_size: int):
    for i in range(len(seq) - window_size + 1):
        yield seq[i: i + window_size]

Coordinate = tuple[int, int]

def get_points_from_anchors(start: Coordinate, end: Coordinate) -> Iterable[Coordinate]:
    """ returns all points in a line between the start and the end """
    def _inner():
        if start == end:
            yield start
            return
        start_x, start_y = start
        end_x, end_y = end
        if start_x == end_x:
            for y in range(min(start_y, end_y), max(start_y, end_y) + 1):
                yield (start_x, y)
        elif start_y == end_y:
            for x in range(min(start_x, end_x), max(start_x, end_x) + 1):
                yield (x, start_y)
    return list(_inner())

def parse_path(path: str):
    anchor_points = []  # an anchor point determines the start or the end of a path
    for anchor_point in path.split(" -> "):
        x, y = anchor_point.split(",")
        anchor_points.append((int(x), int(y)))

    for point_a, point_b in window(anchor_points, 2):
        yield from get_points_from_anchors(point_a, point_b)

def get_stone_points(rows) -> set[Coordinate]:
    stone = set()
    for row in rows:
        for point in parse_path(row):
            stone.add(point)
    return stone

def print_board(stone_points: set[Coordinate], sand_points: set[Coordinate]):
    """ only print the parts of the board where there is sand or stone so that we don't print a huge amount of empty space"""
    (min_x, min_y), (max_x, max_y) = get_min_maxes(stone_points.union(sand_points))

    for y in range(min_y, max_y + 1):
        row_chars = []
        for x in range(min_x, max_x + 1):
            point = (x, y)
            if point in stone_points:
                char = "#"
            elif point in sand_points:
                char = "o"
            else:
                char = "."
            row_chars.append(char)
        print("".join(row_chars))
    print("")

def get_min_maxes(points: set[Coordinate]):
    x_points = [x for (x, y) in points]
    y_points = [y for (x, y) in points]
    mins = min(x_points), min(y_points)
    maxes = max(x_points), max(y_points)
    return mins, maxes



1072


In [16]:
def question_14_a():
    rows = get_puzzle_input_as_rows(2022, 14, test=False)

    stone_points = get_stone_points(rows)
    sand_points = set()

    _, (_, max_stone_y) = get_min_maxes(stone_points)

    # print_board(stone_points, sand_points)  # see starting board

    SAND_ORIGIN = (500, 0)

    into_void = False

    while not into_void:
        sand_moving = True
        sand_x, sand_y = SAND_ORIGIN
        while sand_moving:

            if sand_y > max_stone_y:
                into_void = True
                break

            down_pos = (sand_x, sand_y + 1)
            down_left_pos = (sand_x - 1, sand_y + 1)
            down_right_pos = (sand_x + 1, sand_y + 1)

            # A unit of sand always falls down one step if possible
            if down_pos not in stone_points and down_pos not in sand_points:
                sand_x, sand_y = down_pos

            # the unit of sand attempts to instead move diagonally one step down and to the left
            elif down_left_pos not in stone_points and down_left_pos not in sand_points:
                sand_x, sand_y = down_left_pos

            # If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right
            elif down_right_pos not in stone_points and down_right_pos not in sand_points:
                sand_x, sand_y = down_right_pos

            # If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves
            # at which point the next unit of sand is created back at the source.
            else:
                sand_moving = False
                sand_points.add((sand_x, sand_y))
                # print_board(stone_points, sand_points)

    # print_board(stone_points, sand_points)  # see final board

    print(len(sand_points))

question_14_a()  # 1072

1072
